In [2]:
# Import dependencies
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
# Import datasets
testDataFrame = pd.read_csv("./datasets/mnist/mnistTest.csv")
trainDataFrame = pd.read_csv("./datasets/mnist/mnistTrain.csv")

In [4]:
# Load data into numpy arrays
testData = np.array(testDataFrame)
trainData = np.array(trainDataFrame)

In [5]:
# Define functions
def sigmoid(x):
  return 1 / ( 1 + np.exp(-x))

def sigmoidPrime(x):
  return np.exp(-x) / ( 1 + np.exp(-x)) ** 2

def ReLU(Z):
  return np.maximum(Z, 0)

def ReluPrime(Z):
  return Z > 0

def softmax(Z):
  A = np.exp(Z) / sum(np.exp(Z))
  return A

def error(l, p):
  if (len(p) != len(l)):
    raise ValueError("Lengths must be the same")

  return np.sum([0.5 * (p[i] - l[i])**2 for i in range(len(p))])


### Initialize the weights $w^1_{ij}$ and $ w^2_{kl}$

- $w^0_{ij}$ will be a linear map $w^0: \mathbb{R}^n \to \mathbb{R}^m$

- $w^1_{kl}$ will be a linear map $w^1: \mathbb{R}^m \to \mathbb{R}^l$ 

In [30]:
epsilon = 0.1
labelsRaw = trainData[:,0]
Y = np.array([[1 if label == i else 0 for i in range(10)] for label in labelsRaw])
X = trainData[:,1:] # The input vector
n = 784 # Dimension of the input space (28 * 28 = 784 pixels)
m = 4 # Dimension of hidden layer ( 4 nodes )
l = 10 # Dimension of output ( 10 possible digits to classify )
k = 60000 # Size of training data

def initializeParameters():
  # Initialize weights for the input and the hidden layer
  # Between -0.5 and 0.5 since we are using ReLU as activation function
  W0 = np.random.rand(n, m) - 0.5
  W1 = np.random.rand(m, l) - 0.5

  # Initialize biases
  b0 = np.random.rand(m) - 0.5
  b1 = np.random.rand(l) - 0.5

  return W0, b0, W1, b1

#### Update Bias $b^0$
<details>

$$
\begin{align}
  \frac{\partial \alpha^1_k}{\partial b^0_i} &= \frac{\partial}{\partial b^0_i}
  \bigg (
      \sum_l w^1_{lk} \sigma ( \alpha^0_l ) + b^1_k
  \bigg )
  \\
  & = \sum_l w^1_{lk} \sigma'( \alpha^0_l )  \delta_{il}
  \\
  & =  w^1_{ik} \sigma'( \alpha^0_i )
\end{align}
$$
$$
\Delta b^0_i = - \epsilon \cdot  \sigma'(\alpha^0_i) \cdot \sum_k (p_k - l_k)p_k (1 + p_k) \cdot w^1_{ik}

$$

</details>

#### Update Bias $b^1$
<details>

$$
\begin{align}
  \frac{\partial \alpha^1_k}{\partial b^1_i} &= \frac{\partial}{\partial b^1_i}
  \bigg (
      \sum_l w^1_{lk} \sigma ( \alpha^0_l ) + b^1_k
  \bigg )
  \\
  & =  \delta_{ki}
\end{align}
$$
$$
\Delta b^0_i = - \epsilon  (p_i - l_i)p_i (1 + p_i)

$$

</details>

In [18]:
def forwardProp(W0, W1, b0, b1, X_i):
  # Z is value before activation function is applied
  A0 = W0.T.dot(X_i) + b0
  Z0 = ReLU(A0)
  Z1 = W1.T.dot(Z0) + b1
  P = softmax(Z1)
  return A0, P

def backProp(W1, P, X_i, Y_i, A0):

  def z(k):
    return (P[k] - Y_i[k]) * P[k] * (1 + P[k])

  dW0 = np.array([[ReluPrime(A0[j]) 
    * X_i[j] 
    * np.sum([z(k) * W1[j][k] for k in range(l)]) 
        for i in range(n)] 
          for j in range(m)])

  db0 = np.array([
    ReluPrime(A0[i]) * 
    np.sum([z(k) * W1[i][k] for k in range(l)]) 
      for i in range(m)])

  dW1 = np.array([[ReluPrime(A0[i]) 
    * z(j) 
    * A0[i] 
      for i in range(m)] 
        for j in range(l)])

  db1 = np.array([z(k) for k in range(l)])
  return dW0, db0, dW1, db1



In [31]:
def getPredictions(P):
    return np.argmax(P, 0)

def getAccuracy(p, Y):
  return np.sum(p == Y) / Y.size

def updateParams(W0, W1, b0, b1, dW0, dW1, db0, db1, epsilon):
  W0 = W0 - epsilon * dW0.T
  b0 = b0 - epsilon * db0
  W1 = W1 - epsilon * dW1.T
  b1 = b1 - epsilon * db1

  return W0, b0, W1, b1

#  One iteration
def gradientDescent(X, Y, epsilon, iterations):
  
  W0, b0, W1, b1 = initializeParameters()

  for i in range(iterations):
    A0, P = forwardProp(W0, W1, b0, b1, X[i])

    dW0, db0, dW1, db1 = backProp(W1, P, X[i], Y[i], A0)

    W0, b0, W1, b1 = updateParams(W0, W1, b0, b1, dW0, dW1, db0, db1, epsilon)

  return W0, b0, W1, b1


In [32]:
W0, b0, W1, b1 = gradientDescent( X, Y, epsilon, 100)

Starting W1 [[-0.28054908 -0.14003928  0.26596594  0.36811256  0.44686228 -0.01537834
   0.08931857 -0.07828527 -0.47429504 -0.3851394 ]
 [-0.19415441 -0.08062332 -0.46214348 -0.17816137 -0.23230505 -0.36700384
  -0.06432136  0.27317416  0.14809647  0.38447316]
 [-0.06670169 -0.49022227  0.32728527  0.32881533 -0.28956013 -0.06495044
  -0.37767082 -0.16905379  0.45145509 -0.07394961]
 [-0.44606125 -0.21776379 -0.44164347  0.40781033 -0.19164307  0.04264165
  -0.2292001   0.16745079 -0.35094277  0.11331279]]


/home/dfeng/documents/L2Learn/venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Forward Propogation
<details>

Begin with vector input $v \in \mathbb{R}^n$ where $n = 28 * 28 = 784$. The value at the hidden layer before regularization

$$
  \alpha^0_i =
      \sum_{j} w^0_{ji} v_j + b^0_i
$$

where $w_{ij}$ is the weight of neuron $i$ going into neuron $j$ and $b$ is the bias. Let
 $\sigma$ be  some non linear activation function

The value at the output layer
$$
  \alpha^1_i = \sum_j w^1_{ji} \sigma (\alpha^0_j) + b^1_i
$$

Then we apply a softmax function to get our prediction probabilities at the output layer
$$
  p_i = \text{softmax} (\alpha^1_i)
$$

###  Back Propogation

Define the error function as the squared difference between prediction and labelled value. Let $l_i$ define the label for input $i$
$$
  E(w^0, w^1) = \frac{1}{2} \sum_i (l_i - p_i)^2
$$
We want to minimize $E(w^0, w^1)$. Handle the $w^0$ and $w^1$ cases separately.
</details>


### Hidden layer weights $w^1$
<details>

$$
  \frac{\partial E}{\partial w^1_{ij}} = \sum_k \frac{\partial E}{\partial p_k} \frac{\partial p_k}{\partial \alpha^1_k} \frac{\partial \alpha^1_k}{w^1_{ij}}
$$

Calculate each factor
$$
\begin{aligned}
  \frac{\partial E}{\partial p_k} &= \frac{1}{2} (l_k - p_k) * (-1) \\
  &= (p_k - l_k) \\
  \\
  \frac{\partial p_k}{\partial \alpha^1_k} 
    
    &= \frac{\partial}{\partial \alpha^1_k} 
        \frac{e^{\alpha^1_k}}{\sum_l e^{\alpha^1_l}} 
  \\
  & = e^{\alpha^1_k} 
    \bigg ( 
      \frac{ e^{\alpha^1_k} }{ 
        \big( 
          \sum_l e^{ \alpha^1_k } 
        \big)^2} +
      \frac{1}{ \sum_l e^{ \alpha^1_l } } 
    \bigg ) 
  \\
  & = \frac{ e^{\alpha^1_k}}{\sum_l e^{\alpha^1_l} } 
    \bigg ( 
      1 + \frac{e^{\alpha^1_k}}{\sum_l e^{\alpha^1_l}} 
    \bigg )
  \\
  & = p_k ( 1 + p_k)
  
\end{aligned}
$$
Finally the derivative wrt the weight
$$
\begin{align}
  \frac{ \partial \alpha^1_k }{ \partial w^1_{ij} } 
    &= 
    \frac{\partial}{\partial w^1_{ij}} 
    \bigg( 
      \sum_l w^1_{lk} \sigma ( \alpha^0_l ) + b^1_k
    \bigg)
  \\
&= \sum_l \sigma ' ( \alpha^0_l ) \cdot \delta_{kj} \delta_{il} \alpha_l = \delta_{kj} \sigma ' ( \alpha^0_i ) \alpha_i
\end{align}
$$
Where $\delta$ denotes the Kroneckor delta. Increment the hidden layer output weights proportional to the derivative of the error where $\epsilon$ is some proportionality factor
$$
\begin{align}

  \Delta w^1_{ij} &= - \epsilon \sum_k p_k(p_k - l_k)(1 + p_k) \delta_{kj} \alpha^0_i \sigma ' ( \alpha^0_i )
  \\
  &= - \epsilon p_j(p_j - l_j)(1 + p_j) \alpha^0_i \sigma ' ( \alpha^0_i )
\end{align}
$$

</details>

### Input layer weights $w^0$
<details>

$$
  \frac{ \partial E }{ \partial w^0_{ij} } = 
  \sum_k
    \frac{ \partial E }{ \partial p_k } 
    \frac{ \partial p_k }{ \partial \alpha^1_k } 
    \frac{ \partial \alpha^1_k }{ w^0_{ij} }
$$

Only the last term is different. Keeping $\sigma$ generic:
$$
\begin{align}
  \frac{\partial \alpha^1_k}{\partial w^0_{ij}} 
  
    &= \frac{ \partial }{ \partial w^0_{ij} } 
      \bigg (
        \sum_l w^1_{lk} \sigma (\alpha^0_l) + b^1_k
      \bigg)
    \\

     &= \sum_l 
      w^1_{lk} 
      \cdot
      \frac{ \partial }{ \partial w^0_{ij} }
        \sigma 
          \big( 
            \alpha^0_l
          \big ) 
    \\

    &= \sum_l 
        w^1_{lk} 
        \cdot
        \sigma'(\alpha^0_l)
        \cdot
        \frac{ \partial }{ \partial w^0_{ij} }
          \bigg ( 
            \sum_{m} w^0_{ml} v_m + b^0_l
          \bigg )
    \\
    &= \sum_l
        w^1_{lk}
        \cdot
        \sigma'(\alpha^0_l)
        \cdot
        \sum_m
        \delta_{im} \delta_{jl} v_m 
    \\
    &= \sum_l
        w^1_{lk}
        \cdot
        \sigma'(\alpha^0_l)
        \cdot
        \delta_{jl} v_i
    \\
    &= w^1_{jk}
      \cdot 
      \sigma'(\alpha^0_j)
      \cdot
      v_i
        
\end{align}
$$

Combining the previously calculated terms we get
$$
\begin{align}

  \Delta w^0_{ij}
  &= 
  - \epsilon \cdot \sum_k
    (p_k - l_k)
    p_k
    (1 + p_k) 
    \cdot 
    w^1_{jk}
      \cdot 
      \sigma'(\alpha^0_j)
      \cdot
      v_i
  \\
  &= - \epsilon 
    \cdot \sigma'(\alpha^0_j) 
    v_i 
    \sum_k
     (p_k - l_k)
      p_k
      (1 + p_k) 
      w^1_{jk}
\end{align}
$$

</details>